In [21]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)


#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bellybutton.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples


# Use Pandas to perform the sql query
stmt = db.session.query(Samples_Metadata).statement
df = pd.read_sql_query(stmt, db.session.bind)

# Return a list of the column names (sample names)
df.head()









C:\Users\lorie\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


,sample,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,None,1,NaN,33.990002,25.5
1,941,None,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,None,None,...,None,None,NaN,NaN,NaN,None,None,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,None,None,...,None,None,NaN,NaN,NaN,None,None,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,None,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,None,0,5613.0,33.634445,24.0


In [37]:
sel = [
    Samples_Metadata.sample,
    Samples_Metadata.ETHNICITY,
    Samples_Metadata.GENDER,
    Samples_Metadata.AGE,
    Samples_Metadata.LOCATION,
    Samples_Metadata.BBTYPE,
    Samples_Metadata.WFREQ,
]

results = db.session.query(*sel).filter(Samples_Metadata.sample).all()

# Create a dictionary entry for each row of metadata information
sample_metadata = {}
for result in results:
    sample_metadata["sample"] = result[0]
    sample_metadata["ETHNICITY"] = result[1]
    sample_metadata["GENDER"] = result[2]
    sample_metadata["AGE"] = result[3]
    sample_metadata["LOCATION"] = result[4]
    sample_metadata["BBTYPE"] = result[5]
    sample_metadata["WFREQ"] = result[6]

sample_metadata


{'sample': 1601,
 'ETHNICITY': 'Caucasian',
 'GENDER': 'F',
 'AGE': 22.0,
 'LOCATION': 'NC',
 'BBTYPE': 'I',
 'WFREQ': 2.0}

In [32]:
"""Return `otu_ids`, `otu_labels`,and `sample_values`."""
stmt = db.session.query(Samples).statement
df = pd.read_sql_query(stmt, db.session.bind)

# Filter the data based on the sample number and
# only keep rows with values above 1
sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
# Format the data to send as json
data = {
    "otu_ids": sample_data.otu_id.values.tolist(),
    "sample_values": sample_data[sample].values.tolist(),
    "otu_labels": sample_data.otu_label.tolist(),
}
data

NameError: name 'sample' is not defined